# Environmental Analysis Around Rahima Moosa Hospital
This notebook analyzes Land Surface Temperature (LST) and vegetation patterns around Rahima Moosa Hospital using Earth Engine data.

In [ ]:
import ee
import geemap
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    print("Error initializing Earth Engine. Please authenticate first.")
    print("Run 'earthengine authenticate' in your terminal.")
    raise e

In [ ]:
# Define constants
HOSPITAL_LAT = -26.1752
HOSPITAL_LON = 28.0183
JOBURG_CENTER = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT])
BUFFER_DISTANCE = 5000  # 5km buffer
JOBURG_AREA = JOBURG_CENTER.buffer(BUFFER_DISTANCE)

In [ ]:
def get_landsat_collection(start_year, end_year, months=[9,10,11,12,1,2]):
    """Get Landsat collection for the specified time period."""
    # Use Landsat 8 for recent/future and Landsat 5 for historical
    if start_year >= 2013:
        collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        thermal_band = 'ST_B10'
        mult = 0.00341802
        add = 149.0
    else:
        collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
        thermal_band = 'ST_B6'
        mult = 0.00341802
        add = 149.0
    
    # Filter by date and area
    filtered = collection.filterBounds(JOBURG_AREA) \
        .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
        .filter(ee.Filter.calendarRange(months[0], months[-1], 'month'))
    
    return filtered, thermal_band, mult, add

def calculate_lst(image, thermal_band, mult, add):
    """Calculate Land Surface Temperature."""
    # Convert thermal band to celsius
    lst = image.select(thermal_band).multiply(mult).add(add).subtract(273.15)
    return image.addBands(lst.rename('LST'))

def calculate_ndvi(image):
    """Calculate NDVI."""
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

## Historical Period (1980-1989)

In [ ]:
# Create historical map
Map1 = geemap.Map(center=[HOSPITAL_LAT, HOSPITAL_LON], zoom=13)
Map1.add_basemap('HYBRID')

# Get historical data
collection, thermal_band, mult, add = get_landsat_collection(1980, 1989)
processed = collection.map(lambda img: calculate_lst(img, thermal_band, mult, add)) \
                      .map(calculate_ndvi)

# Calculate mean LST and NDVI
mean_lst = processed.select('LST').mean()
mean_ndvi = processed.select('NDVI').mean()

# Add layers
Map1.addLayer(mean_lst, {'min': 25, 'max': 35, 'palette': ['blue', 'yellow', 'red']}, 'LST (1980-1989)')
Map1.addLayer(mean_ndvi, {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI (1980-1989)')

# Add hospital marker and buffer
circle = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT]).buffer(BUFFER_DISTANCE)
Map1.addLayer(circle, {'color': 'red'}, '5km Buffer')
hospital = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT])
Map1.addLayer(hospital, {'color': 'red'}, 'Rahima Moosa Hospital')

Map1.add_layer_control()
Map1

## Current Period (2015-2024)

In [ ]:
# Create current map
Map2 = geemap.Map(center=[HOSPITAL_LAT, HOSPITAL_LON], zoom=13)
Map2.add_basemap('HYBRID')

# Get current data
collection, thermal_band, mult, add = get_landsat_collection(2015, 2024)
processed = collection.map(lambda img: calculate_lst(img, thermal_band, mult, add)) \
                      .map(calculate_ndvi)

# Calculate mean LST and NDVI
mean_lst = processed.select('LST').mean()
mean_ndvi = processed.select('NDVI').mean()

# Add layers
Map2.addLayer(mean_lst, {'min': 25, 'max': 35, 'palette': ['blue', 'yellow', 'red']}, 'LST (2015-2024)')
Map2.addLayer(mean_ndvi, {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI (2015-2024)')

# Add hospital marker and buffer
circle = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT]).buffer(BUFFER_DISTANCE)
Map2.addLayer(circle, {'color': 'red'}, '5km Buffer')
hospital = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT])
Map2.addLayer(hospital, {'color': 'red'}, 'Rahima Moosa Hospital')

Map2.add_layer_control()
Map2

## Projected Period (2045-2055)

In [ ]:
# Create projected map
Map3 = geemap.Map(center=[HOSPITAL_LAT, HOSPITAL_LON], zoom=13)
Map3.add_basemap('HYBRID')

# Get projected data
collection, thermal_band, mult, add = get_landsat_collection(2045, 2055)
processed = collection.map(lambda img: calculate_lst(img, thermal_band, mult, add)) \
                      .map(calculate_ndvi)

# Calculate mean LST and NDVI
mean_lst = processed.select('LST').mean()
mean_ndvi = processed.select('NDVI').mean()

# Add layers
Map3.addLayer(mean_lst, {'min': 25, 'max': 35, 'palette': ['blue', 'yellow', 'red']}, 'LST (2045-2055)')
Map3.addLayer(mean_ndvi, {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI (2045-2055)')

# Add hospital marker and buffer
circle = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT]).buffer(BUFFER_DISTANCE)
Map3.addLayer(circle, {'color': 'red'}, '5km Buffer')
hospital = ee.Geometry.Point([HOSPITAL_LON, HOSPITAL_LAT])
Map3.addLayer(hospital, {'color': 'red'}, 'Rahima Moosa Hospital')

Map3.add_layer_control()
Map3